## Applied Data Science Capstone Peer-graded Assignment week #3
### Segmenting and Clustering Neighborhoods in Toronto
#### Part One
Transform the data in the table on the Wikipedia page into the a pandas dataframe.

In [1]:
#Install lxml to process HTML with Python
pip install lxml

     |████████████████████████████████| 5.8MB 20.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
import numpy as np
import lxml.html as lh

In [5]:
#Wiki data page url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Create a handle, page, to handle the contents of the wikipage
wikipage = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(wikipage.content)

In [20]:
 #Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#First row is the header, last 5 tr elements are from other tables, 
#Data is stored on the second row onwards to 288

#Create column list
col = [('Postcode', []), ('Borough', []), ('Neighbourhood', [])]
for j in range(1, len(tr_elements)):
    rows = tr_elements[j]

    #If row is not of size 3, the //tr data is not from our table 
    if len(rows) != 3:
        break
    
    #i is the index of the column
    i = 0
    
    #Iterate through each element of the row
    for r in rows.iterchildren():
        data = r.text_content().replace('\n', '') 
        col[i][1].append(data)
        i += 1   

#Creat pandas dataframe
df = {title:column for (title, column) in col}
df = pd.DataFrame(df)

print(df)
df.shape

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
283      M8Z         Etobicoke              Mimico NW
284      M8Z         Etobicoke     The Queensway West
285      M8Z         Etobicoke  Royal York South West
286      M8Z         Etobicoke         South of Bloor
287      M9Z      Not assigned           Not assigned

[288 rows x 3 columns]


(288, 3)

In [21]:
#Drop not assigned Borough
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
print(df)

    Postcode           Borough             Neighbourhood
0        M3A        North York                 Parkwoods
1        M4A        North York          Victoria Village
2        M5A  Downtown Toronto              Harbourfront
3        M5A  Downtown Toronto               Regent Park
4        M6A        North York          Lawrence Heights
..       ...               ...                       ...
206      M8Z         Etobicoke  Kingsway Park South West
207      M8Z         Etobicoke                 Mimico NW
208      M8Z         Etobicoke        The Queensway West
209      M8Z         Etobicoke     Royal York South West
210      M8Z         Etobicoke            South of Bloor

[211 rows x 3 columns]


In [22]:
#Set not assigned Neighbourhood with Borough
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough
print(df)

    Postcode           Borough             Neighbourhood
0        M3A        North York                 Parkwoods
1        M4A        North York          Victoria Village
2        M5A  Downtown Toronto              Harbourfront
3        M5A  Downtown Toronto               Regent Park
4        M6A        North York          Lawrence Heights
..       ...               ...                       ...
206      M8Z         Etobicoke  Kingsway Park South West
207      M8Z         Etobicoke                 Mimico NW
208      M8Z         Etobicoke        The Queensway West
209      M8Z         Etobicoke     Royal York South West
210      M8Z         Etobicoke            South of Bloor

[211 rows x 3 columns]


In [23]:
#For same Postcode, combine Neighbourhood into one row with the neighborhoods separated with a comma
neighbor_df = df.groupby(['Postcode', 'Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
neighbor_df = neighbor_df.reset_index()
print(neighbor_df)

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                     Rouge, Malvern
1        M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                  Guildwood, Morningside, West Hill
3        M1G  Scarborough                                             Woburn
4        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
98       M9N         York                                             Weston
99       M9P    Etobicoke                                          Westmount
100      M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101      M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102      M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


<hr />

#### Part Two
Use the Geocoder package or the csv file to create the required dataframe.

In [24]:
#Load Geospatial Coordinates from csv file
filename = "Geospatial_Coordinates.csv"
headers = ["Postcode","Latitude","Longitude"]
geo_df = pd.read_csv(filename, names = headers)
geo_df = geo_df.drop(geo_df.index[0])
print(geo_df)
print(geo_df.shape)

    Postcode    Latitude    Longitude
1        M1B  43.8066863  -79.1943534
2        M1C  43.7845351  -79.1604971
3        M1E  43.7635726  -79.1887115
4        M1G  43.7709921  -79.2169174
5        M1H   43.773136  -79.2394761
..       ...         ...          ...
99       M9N   43.706876  -79.5181884
100      M9P   43.696319  -79.5322424
101      M9R  43.6889054  -79.5547244
102      M9V  43.7394164  -79.5884369
103      M9W  43.7067483  -79.5940544

[103 rows x 3 columns]
(103, 3)


In [25]:
#Combine two pandas data frame on Postcode
neighbourhood_df = pd.merge(neighbor_df, geo_df, on='Postcode', how='inner')
neighbourhood_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.2620294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.2648481
